In [ ]:
!git clone https://github.com/WittmannF/sort-google-scholar.git

fatal: destination path 'sort-google-scholar' already exists and is not an empty directory.


In [ ]:
!wget https://chromedriver.storage.googleapis.com/2.42/chromedriver_mac64.zip  && unzip chromedriver_mac64.zip

--2021-03-17 00:23:08--  https://chromedriver.storage.googleapis.com/2.42/chromedriver_mac64.zip
Resolving chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)... 173.194.69.128, 2a00:1450:4013:c04::80
Connecting to chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)|173.194.69.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6025563 (5.7M) [application/zip]
Saving to: ‘chromedriver_mac64.zip.1’

chromedriver_mac64. 100%[===================>]   5.75M  --.-KB/s    in 0.04s   

2021-03-17 00:23:08 (134 MB/s) - ‘chromedriver_mac64.zip.1’ saved [6025563/6025563]

Archive:  chromedriver_mac64.zip
replace chromedriver? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!pip install selenium

In [ ]:
! pip install chromedriver-py


In [ ]:
# Change 'deep learning' with the keyword that you would like to check
chromedriver_path= 'chromedriver'
!python /content/sort-google-scholar/sortgs.py --kw "social movement theory" --sortby "cit/year"

> _**NOTE:** It is normal to get some warnings, for example year not found or author not found. However, if you get the robot checking warning, then it might not work anymore in the IP that you have on Google Colab. You can try going in 'Runtime' and 'Reset all runtimes' to get a new IP, but it is not garanteed that it will work. If the problem persists, then you will have to run locally using selenium and solve the captchas manually. Make sure to avoid running this code too often to avoid the robot checking problem._

Next, you will see that a csv file with the name of the keyword was created (which is `deep_learning.csv` in my case). 

In [ ]:
ls

Let's import this file to visualize some results (make sure to change `deep_learning.csv` to the file name that was created in your case):

In [ ]:
import pandas as pd
df = pd.read_csv('social_movement_theory.csv') # OBS: Update the file name

In [ ]:
df.head(10)

Finally, let's download the CSV results (make sure to update the CSV name here as well):

In [ ]:
from google.colab import files
files.download('social_movement_theory.csv') # Update the file name here

After selecting top chosen articles, pull the text from the PDF files using https://www.pdf2go.com/pdf-to-text and input text into BART summarizer.

In [ ]:
!git clone https://github.com/huggingface/transformers \
&& cd transformers \

In [ ]:
!pip install -q ./transformers

In [ ]:
import torch
import transformers
from transformers import BartTokenizer, BartForConditionalGeneration

In [ ]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

In [ ]:
#torch_device = 'cuda' if torch.cuda.is_available() else 'cpu '# failing when device is gpu
torch_device = 'cpu'

In [ ]:
import google.colab.output

def bart_summarize(text, num_beams, length_penalty, max_length, min_length, no_repeat_ngram_size):
  
  text = text.replace('\n','')
  text_input_ids = tokenizer.batch_encode_plus([text], return_tensors='pt', max_length=1024)['input_ids'].to(torch_device)
  summary_ids = model.generate(text_input_ids, num_beams=int(num_beams), length_penalty=float(length_penalty), max_length=int(max_length), min_length=int(min_length), no_repeat_ngram_size=int(no_repeat_ngram_size))           
  summary_txt = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)
  return summary_txt

#register callback for Javascript
google.colab.output.register_callback('bart_summarize', bart_summarize)

In [ ]:
from IPython.display import HTML

#spinner from https://codepen.io/vovchisko/pen/vROoYQ
spinner_css = """
<style>
@keyframes c-inline-spinner-kf {
  0% {
    transform: rotate(0deg);
  }
  100% {
    transform: rotate(360deg);
  }
}

.c-inline-spinner,
.c-inline-spinner:before {
  display: inline-block;
  width: 11px;
  height: 11px;
  transform-origin: 50%;
  border: 2px solid transparent;
  border-color: #74a8d0 #74a8d0 transparent transparent;
  border-radius: 50%;
  content: "";
  animation: linear c-inline-spinner-kf 300ms infinite;
  position: relative;
  vertical-align: inherit;
  line-height: inherit;
}
.c-inline-spinner {
  top: 3px;
  margin: 0 3px;
}
.c-inline-spinner:before {
  border-color: #74a8d0 #74a8d0 transparent transparent;
  position: absolute;
  left: -2px;
  top: -2px;
  border-style: solid;
}
</style>
"""

input_form = """
<link rel="stylesheet" href="https://unpkg.com/purecss@1.0.1/build/pure-min.css" integrity="sha384-oAOxQR6DkCoMliIh8yFnu25d7Eq/PHS21PClpwjOTeU2jRSq11vu66rf90/cZr47" crossorigin="anonymous">

<div style="background-color:white; border:solid #ccc; width:800px; padding:20px; color: black;">
<p><strong>BART</strong> Seq2Seq model with SoTA summarization performance</p>
<textarea id="main_textarea" cols="75" rows="20" placeholder="Paste your text here..." style="font-family: 'Liberation Serif', 'DejaVu Serif', Georgia, 'Times New Roman', Times, serif; font-size: 13pt; padding:10px;"></textarea><br>
<div class="pure-form pure-form-aligned">
   <div class="pure-control-group">
     <label for="no_repeat_ngram_size"><strong>no_repeat_ngram_size:</strong></label>
     <input type="number" id="no_repeat_ngram_size" value="3" style="background-color: white;">
    </div>
    <div class="pure-control-group">
      <label for="num_beams"><strong>num_beams:</strong></label>
      <input type="number" min="0" max="10" step="1" id="num_beams" value="4" style="background-color: white;">
    </div>
    <div class="pure-control-group">
        <label for="length_penalty"><strong>length_penalty:</strong></label>
        <input type="number" min="0.0" max="10.0" step="0.1" id="length_penalty" value="2.0" style="background-color: white;">
    </div>
    <div class="pure-control-group">
        <label for="max_length"><strong>max_length:</strong></label>
        <input type="number" id="max_length" value="142" style="background-color: white;">
    </div>
     <div class="pure-control-group">
        <label for="min_length"><strong>min_length:</strong></label>
        <input type="number" id="min_length" value="56" style="background-color: white;">
    </div>
    <p><a target="_blank" href='https://pastebin.com/raw/BMPcUS6v'>Try to summarize this example article</a></p>
    <div style="width: 300px; display: block; margin-left: auto !important; margin-right: auto !important;">
        <p><button class="pure-button pure-button-primary" style="font-size: 125%%;" onclick="summarize()">Summarize</button>
        <span class="c-inline-spinner" style="visibility: hidden;" id="spinner"></span></p>
    </div>
</div>
</div>
"""

javascript = """
<script type="text/Javascript">


       function saveTextAsFile(textToWrite, fileNameToSaveAs)
    {
    	var textFileAsBlob = new Blob([textToWrite], {type:'text/plain'}); 
    	var downloadLink = document.createElement("a");
    	downloadLink.download = fileNameToSaveAs;
    	downloadLink.innerHTML = "Download File";
    	if (window.webkitURL != null)
    	{
    		// Chrome allows the link to be clicked
    		// without actually adding it to the DOM.
    		downloadLink.href = window.webkitURL.createObjectURL(textFileAsBlob);
    	}
    	else
    	{
    		// Firefox requires the link to be added to the DOM
    		// before it can be clicked.
    		downloadLink.href = window.URL.createObjectURL(textFileAsBlob);
    		downloadLink.onclick = destroyClickedElement;
    		downloadLink.style.display = "none";
    		document.body.appendChild(downloadLink);
    	}
    
    	downloadLink.click();
    }


    function summarize(){
        
        var text = document.getElementById('main_textarea').value;
        var no_repeat_ngram_size = document.getElementById('no_repeat_ngram_size').value;
        var num_beams = document.getElementById('num_beams').value;
        var length_penalty = document.getElementById('length_penalty').value;
        var max_length = document.getElementById('max_length').value;
        var min_length = document.getElementById('min_length').value;
        
        var kernel = google.colab.kernel;

        var resultPromise = kernel.invokeFunction("bart_summarize", [text,num_beams,length_penalty,max_length,min_length,no_repeat_ngram_size]); // developer, look here
        resultPromise.then(
            function(result) {
              document.getElementById('main_textarea').value = 'da resultado';
              document.getElementById('main_textarea').value = result.data["text/plain"];
              document.getElementById('spinner').style = "visibility: hidden;";
              saveTextAsFile(result.data["text/plain"], 'summary.txt')
        }).catch(function(error){document.getElementById('main_textarea').value = error;});
        document.getElementById('spinner').style = "visibility: visible;";
    };
</script>
""" 


HTML(spinner_css + input_form + javascript)